# Group Emotion Recognition

## 1. Data Preprocessing

In [1]:
import cv2
import os
from PIL import Image
import glob
import numpy as np

### 1.1 Face and Label Detection

### 1.1.1 Using Google Vision API

**Import Google Vision Library**

In [ ]:
from google.cloud import vision

**Implement Face Detection function**

In [ ]:
def detect_face_google(face_file, max_results=4):
    """Uses the Vision API to detect faces in the given file.

    Args:
        face_file: A file-like object containing an image with faces.

    Returns:
        An array of Face objects with information about the picture.
    """
    client = vision.ImageAnnotatorClient()

    content = face_file.read()
    image = vision.types.Image(content=content)

    return client.face_detection(image=image).face_annotations

**Implement Label Detection function**

In [ ]:
def detect_labels_google(path):
    """Detects labels in the file."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.label_detection(image=image)
    labels = response.label_annotations
    print('Labels:')

    for label in labels:
        print(label.description)
    
    labels_list = [label.description for label in labels]
    
    return labels_list

---

### 1.1.2. Using OpenCV DNN

In [3]:
def detect_faces(face_file):
    # load our serialized model from disk
    print('[INFO] loading model...')
    net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt',
                                   'res10_300x300_ssd_iter_140000.caffemodel'
                                   )

    # load the input image and construct an input blob for the image
    # by resizing to a fixed 300x300 pixels and then normalizing it
    image = cv2.imread(face_file)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300,
                                 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the detections and
    # predictions
    print('[INFO] computing object detections...')
    net.setInput(blob)
    detections = net.forward()

    return detections

---

### 1.2 Preprocessing

In [ ]:
def crop_faces(image_file, cropped_images_path, faces):
    count = 1
    image = Image.open(image_file)
    
    if not faces:
        print("No face detected in the image.")
        return
    
#     print(len(faces))
    
    for face in faces:
        coordinates = [(vertex.x, vertex.y)
            for vertex in face.bounding_poly.vertices]
        
        x_coordinates, y_coordinates = [], []
        for vertex in face.bounding_poly.vertices:
            x_coordinates.append(vertex.x)
            y_coordinates.append(vertex.y)
        
        x0, x1, y0, y1 = x_coordinates[0], x_coordinates[2], y_coordinates[0], y_coordinates[2]
        
        box = (x0, y0, x1, y1)
        cropped_image = image.crop(box)
                                                              
        image_name = (image_file.split("/")[-1])[:-4]
        cropped_image.save(cropped_images_path + image_name + "_face_" + str(count) + ".jpg")
        count+=1

In [ ]:
def resize_faces(cropped_images_path, scaled_images_path, size):
    count = 1
    for file in glob.glob(cropped_images_path+"*.jpg"):
        image = cv2.imread(file)
        height, width = image.shape[:2]
        
        height_ratio, width_ratio = float(size/height), float(size/width)

        resized = cv2.resize(image, None, fx=width_ratio, fy=height_ratio, interpolation=cv2.INTER_AREA)
        
#         print(file)
        image_name = (file.split("/")[-1])
        cv2.imwrite(scaled_images_path + image_name, resized)

In [ ]:
images = [["emotiw/train/Positive/", "Faces/train/Positive/", "Scaled/train/Positive/"], ["emotiw/val/Positive/", "Faces/val/Positive/", "Scaled/val/Positive/"], ["emotiw/val/Neutral/", "Faces/val/Neutral/", "Scaled/val/Neutral/"], ["emotiw/val/Negative/", "Faces/val/Negative/", "Scaled/val/Negative/"], ["emotiw/train/Neutral/", "Faces/train/Neutral/", "Scaled/train/Neutral/"], ["emotiw/train/Negative/", "Faces/train/Negative/", "Scaled/train/Negative/"]]

for image_category in images:
    for image_file in sorted(glob.glob(image_category[0]+"*.jpg")):
        print(image_file)
        with open(image_file, 'rb') as image:
            faces = detect_face_google(image)

            # Reset the file pointer, so we can read the file again
            image.seek(0)
            crop_faces(image_file, image_category[1], faces)
        break
    break

    resize_faces(image_category[1], image_category[2], 64)